<a href="https://colab.research.google.com/github/ste-corizzato/NLP-Project/blob/main/Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dataset:**

* Controllo Dataset
* Elimino campi inutili
* Ordino per rating
* Check che ci siano tutti i campi che mi interessano
* Pulizia singoli campi

Preso da Kaggle: https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews

In [ ]:
from google.colab import drive
import pandas as pd

## Download Dataset

In [ ]:
! pip install kaggle

In [ ]:
gdrive_dir = '/content/gdrive/'
data_dir = os.path.join(gdrive_dir, "My Drive","NLP","Project(P2)")

In [ ]:
drive.mount(gdrive_dir)

Mounted at /content/gdrive/


In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/gdrive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download irkaal/foodcom-recipes-and-reviews

Dataset URL: https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews
License(s): CC0-1.0
 99% 713M/723M [00:11<00:00, 66.8MB/s]
100% 723M/723M [00:11<00:00, 66.4MB/s]


In [ ]:
! unzip foodcom-recipes-and-reviews.zip

Archive:  foodcom-recipes-and-reviews.zip
  inflating: recipes.csv             
  inflating: recipes.parquet         
  inflating: reviews.csv             
  inflating: reviews.parquet         


In [ ]:
df_recipes = pd.read_parquet("recipes.parquet")

##Cleaning

In [ ]:
for k,l in df_recipes[:10].iterrows():
  print(l['Name'],' ---- ',l['RecipeServings'],' ---- ',l['RecipeYield'])
  #RecipeServings indica per quante persone è
  #RecipeYield indica la quantità di cibo (1 torta, 4 kebabs, etc)

Low-Fat Berry Blue Frozen Dessert  ----  4.0  ----  None
Biryani  ----  6.0  ----  None
Best Lemonade  ----  4.0  ----  None
Carina's Tofu-Vegetable Kebabs  ----  2.0  ----  4 kebabs
Cabbage Soup  ----  4.0  ----  None
Best Blackbottom Pie  ----  8.0  ----  1 9-inch pie
Warm Chicken A La King  ----  2.0  ----  None
Buttermilk Pie With Gingersnap Crumb Crust  ----  8.0  ----  None
A Jad - Cucumber Pickle  ----  nan  ----  1 cup
Butter Pecan Cookies  ----  nan  ----  84 cookies


In [ ]:
print(len(df_recipes))

522517


**The recipe dataset is made up of 500,000 records with 28 fields.**

Tengo: 'RecipeId', 'RecipeIngredientParts', 'AggregatedRating', 'ReviewCount', 'RecipeInstructions'

In [ ]:
df_recipes_clean = df_recipes[['RecipeId','RecipeIngredientParts','AggregatedRating','ReviewCount','RecipeInstructions']].copy()

In [ ]:
for col, j in df_recipes_clean[:2].iterrows():
  print(j)

RecipeId                                                              38.0
RecipeIngredientParts    [blueberries, granulated sugar, vanilla yogurt...
AggregatedRating                                                       4.5
ReviewCount                                                            4.0
RecipeInstructions       [Toss 2 cups berries with sugar., Let stand fo...
Name: 0, dtype: object
RecipeId                                                              39.0
RecipeIngredientParts    [saffron, milk, hot green chili peppers, onion...
AggregatedRating                                                       3.0
ReviewCount                                                            1.0
RecipeInstructions       [Soak saffron in warm milk for 5 minutes and p...
Name: 1, dtype: object


In [ ]:
#I wanna use only the 100'000 best recipes --> i select the recipes with more than 1 ReviewCount and then i order them by AggregateRating
df_rcp_filtered = df_recipes_clean[df_recipes_clean['ReviewCount'] > 1].copy()

In [ ]:
len(df_rcp_filtered)

167973

In [ ]:
#I have to check if the fields "RecipeIngredientParts" and "RecipeInstructions" are filled

df_rcp_filtered = df_rcp_filtered[df_rcp_filtered['RecipeIngredientParts'].str.len() > 0].copy()
df_rcp_filtered = df_rcp_filtered[df_rcp_filtered['RecipeInstructions'].str.len() > 0].copy()

In [ ]:
len(df_rcp_filtered)

167302

In [ ]:
#ordino per Rating e numero di Review
df_rcp_filtered = df_rcp_filtered.sort_values(by=['AggregatedRating', 'ReviewCount'], ascending=False)

In [ ]:
df_final = df_rcp_filtered[:100000]

## Fields Cleaning

In [ ]:
df_final.iloc[1]['RecipeIngredientParts']

#è un array per questo mi sbagliava quando univo ingredienti e istruzione

array(['butter', 'granulated sugar', 'eggs', 'bananas',
       'all-purpose flour', 'baking soda', 'salt', 'vanilla'],
      dtype=object)

In [ ]:
df_final['RecipeIngredientParts'] = df_final['RecipeIngredientParts'].apply(lambda x: ', '.join(x))

<ipython-input-42-22d64c9a1138>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['RecipeIngredientParts'] = df_final['RecipeIngredientParts'].apply(lambda x: ', '.join(x))


In [ ]:
df_final['RecipeInstructions'] = df_final['RecipeInstructions'].apply(lambda x: ', '.join(x))

<ipython-input-43-e95d136f619e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['RecipeInstructions'] = df_final['RecipeInstructions'].apply(lambda x: ', '.join(x))


In [ ]:
df_final.iloc[1]['RecipeInstructions']

'Remove odd pots and pans from oven., Preheat oven to 350º / 180º., Cream together butter and sugar., Add eggs and crushed bananas., Combine well., Sift together flour, soda and salt. Add to creamed mixture. Add vanilla., Mix just until combined. Do not overmix., Pour into greased and floured loaf pan., Bake at 350º / 180º for 55 minutes., Keeps well, refrigerated.'

In [ ]:
df_lil_final = df_final[:10000]
df_medium_final = df_final[:20000]

In [ ]:
df_final.to_csv('/content/gdrive/MyDrive/NLP/Project(P2)/df_final.csv')
df_lil_final.to_csv('/content/gdrive/MyDrive/NLP/Project(P2)/df_lil_final.csv')
df_medium_final.to_csv('/content/gdrive/MyDrive/NLP/Project(P2)/df_medium_final.csv')